In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
if 'google.colab' in str(get_ipython()):
    from google.colab import files
    
    files.upload()

Saving data.zip to data.zip
Saving deepcomedy.zip to deepcomedy.zip


In [ ]:
!pip install wandb
#!tar zxvf deepcomedy.tar.gz
!unzip deepcomedy.zip
#!tar zxvf data.tar.gz
!unzip data.zip

     |████████████████████████████████| 1.8MB 2.9MB/s 
     |████████████████████████████████| 102kB 8.5MB/s 
     |████████████████████████████████| 133kB 20.6MB/s 
     |████████████████████████████████| 174kB 18.6MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=31caef3ebe835e319122388e5730ab7480fb41ed2168193ce589c2b961ebee61
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=89037e1e0b2349eaff16e42db55d022ef529ed05ab918fc329da3de263732c2b
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
Archive:  deepcomedy.zip
   creating: deepcomedy/models/
  inflating: deepcomedy/models/layers.py  
  inflating: deepcomedy/models/transformer.py  
  

In [ ]:
import io
import os
import re
import time
import unicodedata

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental import preprocessing

from deepcomedy.models.transformer import *
from deepcomedy.preprocessing import *
from itertools import chain

## 1. Data preprocessing

In [ ]:
raw_text = open("./data/divina_textonly.txt", "rb").read().decode(encoding="utf-8")
raw_syll_text = (
    open("./data/divina_syll_textonly.txt", "rb").read().decode(encoding="utf-8")
)
syll_text = preprocess_text(raw_syll_text, end_of_verse = "")
text = preprocess_text(raw_text, end_of_verse = "")

Split preprocessed text into verses

In [ ]:
sep = "<EOT>"
input_tercets = [x + sep for x in text.split(sep)][:-1]
target_tercets = [x + sep for x in syll_text.split(sep)][:-1]

Encode with tokenizer

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    char_level=False, filters="", lower=False
)
tokenizer.fit_on_texts(target_tercets)
enc_input_tercets = tokenizer.texts_to_sequences(input_tercets)
enc_target_tercets = tokenizer.texts_to_sequences(target_tercets)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
input_text = []
target_text = []

for line in range(len(enc_input_tercets) - 2):
    input_text.append(list(chain(*enc_input_tercets[line : line + 2])))
    target_text.append(list(chain(*enc_target_tercets[line + 1 : line + 3])))

Pad sequences

In [ ]:
padded_input_text = tf.keras.preprocessing.sequence.pad_sequences(
    input_text, padding="post"
)
padded_target_text = tf.keras.preprocessing.sequence.pad_sequences(
    target_text, padding="post"
)

In [ ]:
input_train, input_test, target_train, target_test = train_test_split(
    padded_input_text, padded_target_text
)

## 2. The Transformer model


In [ ]:
def make_dataset(input_train, target_train, batch_size=32):
    buffer_size = len(input_train)

    dataset = tf.data.Dataset.from_tensor_slices((input_train, target_train)).shuffle(buffer_size)
    dataset = dataset.batch(batch_size, drop_remainder=True)

    return dataset

In [ ]:
dataset = make_dataset(input_train, target_train)

In [ ]:
def make_transformer_model(config, input_vocab_size, target_vocab_size, checkpoint_save_path = None):
    transformer = Transformer(
        num_layers=config["num_layers"],
        d_model=config["d_model"],
        num_heads=config["num_heads"],
        dff=config["dff"],
        input_vocab_size=input_vocab_size,
        target_vocab_size=target_vocab_size,
        pe_input=1000,
        pe_target=1000,
        rate=0.1,
    )
    transformer_trainer = TransformerTrainer(transformer, checkpoint_save_path= checkpoint_save_path)

    return transformer, transformer_trainer

In [ ]:
config = {
    "num_layers" : 6,
    "d_model" : 256,
    "num_heads" : 8,
    "dff" : 1024
}

checkpoint_save_path = "./checkpoints/char-level_gen"

In [ ]:
transformer, transformer_trainer = make_transformer_model(config, vocab_size, vocab_size, checkpoint_save_path= checkpoint_save_path)

## 3. Training

In [ ]:
transformer_trainer.train(dataset, 50)

Epoch 1 Batch 0 Loss 5.3591 Accuracy 0.0009
Epoch 1 Batch 50 Loss 4.1508 Accuracy 0.1494
Epoch 1 Batch 100 Loss 3.6142 Accuracy 0.1795
Epoch 1 Loss 3.5534 Accuracy 0.1830
Time taken for 1 epoch: 153.76 secs

Epoch 2 Batch 0 Loss 2.9743 Accuracy 0.2160
Epoch 2 Batch 50 Loss 2.8866 Accuracy 0.2321
Epoch 2 Batch 100 Loss 2.6464 Accuracy 0.2682
Epoch 2 Loss 2.6055 Accuracy 0.2743
Time taken for 1 epoch: 134.99 secs

Epoch 3 Batch 0 Loss 2.1914 Accuracy 0.3351
Epoch 3 Batch 50 Loss 2.1387 Accuracy 0.3418
Epoch 3 Batch 100 Loss 2.0983 Accuracy 0.3467
Epoch 3 Loss 2.0907 Accuracy 0.3477
Time taken for 1 epoch: 134.40 secs

Epoch 4 Batch 0 Loss 2.0155 Accuracy 0.3615
Epoch 4 Batch 50 Loss 2.0054 Accuracy 0.3582
Epoch 4 Batch 100 Loss 1.9949 Accuracy 0.3597
Epoch 4 Loss 1.9920 Accuracy 0.3602
Time taken for 1 epoch: 134.51 secs

Epoch 5 Batch 0 Loss 1.9939 Accuracy 0.3549
Epoch 5 Batch 50 Loss 1.9572 Accuracy 0.3673
Epoch 5 Batch 100 Loss 1.9522 Accuracy 0.3685
Saving checkpoint for epoch 5 at 

KeyboardInterrupt: ignored

## 4. Generation

In [ ]:
def generate_greedy(encoder_input, decoder_input):

    # encoder_input = tf.convert_to_tensor(encoder_input)
    encoder_input = tf.expand_dims(encoder_input, 0)

    # decoder_input = tf.convert_to_tensor(decoder_input)
    output = tf.expand_dims(decoder_input, 0)
    result = "<GO> "
    tokenized_result = [tokenizer.word_index["<GO>"]]

    for i in range(200):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
            encoder_input, output
        )

        # predictions.shape == (batch_size, seq_len, vocab_size)
        predictions, attention_weights = transformer(
            encoder_input,
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask,
        )

        # select the last character from the seq_len dimension
        predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

        predicted_id = tf.argmax(predictions, axis=-1)

        # concatenate the predicted_id to the output which is given to the decoder as its input.
        output = tf.concat(
            [tf.cast(output, dtype=tf.int32), tf.cast(predicted_id, dtype=tf.int32)],
            axis=-1,
        )
        result += tokenizer.index_word[predicted_id.numpy()[0][0]] + " "
        tokenized_result.append(predicted_id.numpy()[0][0])

        if predicted_id == tokenizer.word_index["<EOT>"]:
            return result, tokenized_result

In [ ]:
def generate_topk(encoder_input, decoder_input, k=5, temperature=0.5):

    encoder_input = tf.expand_dims(encoder_input, 0)

    output = tf.expand_dims(decoder_input, 0)

    result = "<GO> "
    tokenized_result = [tokenizer.word_index["<GO>"]]

    for i in range(200):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
            encoder_input, output
        )

        # predictions.shape == (batch_size, seq_len, vocab_size)
        predictions, attention_weights = transformer(
            encoder_input,
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask,
        )

        # select the last character from the seq_len dimension
        predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)
        predictions, indices = tf.math.top_k(predictions, k=k)

        predictions /= temperature
        predictions = np.squeeze(predictions, axis=0)
        indices = np.squeeze(indices, axis=0)
        indices = np.squeeze(indices, axis=0)
        pred = tf.random.categorical(predictions, num_samples=1)
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        predicted_id = indices[predicted_id]

        predicted_id = tf.expand_dims(predicted_id, 0)
        predicted_id = tf.expand_dims(predicted_id, 0)
        output = tf.concat([output, predicted_id], axis=-1)

        result += tokenizer.index_word[predicted_id.numpy()[0][0]] + " "
        tokenized_result.append(predicted_id.numpy()[0][0])

        if predicted_id == tokenizer.word_index["<EOT>"]:
            return result, tokenized_result

In [ ]:
def clean(x):
  x = re.sub(r'\| \b', '', x)
  x = re.sub(r'\b \|', '', x)
  x = re.sub(r'\|', '', x)
  x = re.sub(r'[ ]+', ' ', x)
  return x

Feeding the encoder the last output

In [ ]:
encoder_input = [tokenizer.word_index["<GO>"]]
decoder_input = [tokenizer.word_index["<GO>"]]

generated_text, generated_tokenized = generate_greedy(encoder_input, decoder_input)
print(generated_text)
print(clean(generated_text))

<GO> | e <SEP> | q u e l | l a <SEP> | c h e <SEP> | s i <SEP> | p i a n | t e <SEP> | d i | s c e r | n a | t a <GO> | d i <SEP> | q u e l | l a <SEP> | c h e <SEP> | p i ù <SEP> | d i <SEP> | l a | s c i a | t a <SEP> | s p e | r a <GO> | d i <SEP> | q u e l | l a <SEP> | c h e <SEP> | p i ù <SEP> | d i <SEP> | l u | c i <SEP> | s i <SEP> | p i a | t a . <EOT> 
<GO> e <SEP> q u e l l a <SEP> c h e <SEP> s i <SEP> p i a n t e <SEP> d i s c e r n a t a <GO> d i <SEP> q u e l l a <SEP> c h e <SEP> p i ù <SEP> d i <SEP> l a s c i a t a <SEP> s p e r a <GO> d i <SEP> q u e l l a <SEP> c h e <SEP> p i ù <SEP> d i <SEP> l u c i <SEP> s i <SEP> p i a t a . <EOT> 


In [ ]:
tokenized_generated = tokenizer.texts_to_sequences([clean(generated_text)])
print(tokenized_generated)

[14, 3, 2, 26, 15, 3, 9, 9, 4, 2, 12, 22, 3, 2, 11, 5, 2, 17, 5, 4, 7, 10, 3, 2, 13, 5, 11, 12, 3, 8, 7, 4, 10, 4, 14, 13, 5, 2, 26, 15, 3, 9, 9, 4, 2, 12, 22, 3, 2, 17, 5, 31, 2, 13, 5, 2, 9, 4, 11, 12, 5, 4, 10, 4, 2, 11, 17, 3, 8, 4, 14, 13, 5, 2, 26, 15, 3, 9, 9, 4, 2, 12, 22, 3, 2, 17, 5, 31, 2, 13, 5, 2, 9, 15, 12, 5, 2, 11, 5, 2, 17, 5, 4, 10, 4, 25, 24]


In [ ]:
generated_text_2, _ = generate_greedy(tokenized_generated[0], decoder_input)
print(generated_text_2)
print(clean(generated_text_2))

<GO> | c h é <SEP> | l a <SEP> | p i ù <SEP> | d i | s t a n | t a <SEP> | p i ù <SEP> | d i | s t a n | t a <GO> | d i <SEP> | q u e l | l a <SEP> | c h e <SEP> | p i ù <SEP> | d i <SEP> | q u e l | l a <SEP> | c h e <SEP> | p i ù <SEP> | p i a n | t a <GO> | c h e <SEP> | p i ù <SEP> | d i <SEP> | s é <SEP> | l a <SEP> | p i a n | g e <SEP> | s i <SEP> | s p i | g a n | t a . <EOT> 
<GO> c h é <SEP> l a <SEP> p i ù <SEP> d i s t a n t a <SEP> p i ù <SEP> d i s t a n t a <GO> d i <SEP> q u e l l a <SEP> c h e <SEP> p i ù <SEP> d i <SEP> q u e l l a <SEP> c h e <SEP> p i ù <SEP> p i a n t a <GO> c h e <SEP> p i ù <SEP> d i <SEP> s é <SEP> l a <SEP> p i a n g e <SEP> s i <SEP> s p i g a n t a . <EOT> 


Feeding the decoder the last output

In [ ]:
generated_text_3, _ = generate_greedy(encoder_input, generated_tokenized)
print(generated_text_2)
print(clean(generated_text_2))

<GO> | c h é <SEP> | l a <SEP> | p i ù <SEP> | d i | s t a n | t a <SEP> | p i ù <SEP> | d i | s t a n | t a <GO> | d i <SEP> | q u e l | l a <SEP> | c h e <SEP> | p i ù <SEP> | d i <SEP> | q u e l | l a <SEP> | c h e <SEP> | p i ù <SEP> | p i a n | t a <GO> | c h e <SEP> | p i ù <SEP> | d i <SEP> | s é <SEP> | l a <SEP> | p i a n | g e <SEP> | s i <SEP> | s p i | g a n | t a . <EOT> 
<GO> c h é <SEP> l a <SEP> p i ù <SEP> d i s t a n t a <SEP> p i ù <SEP> d i s t a n t a <GO> d i <SEP> q u e l l a <SEP> c h e <SEP> p i ù <SEP> d i <SEP> q u e l l a <SEP> c h e <SEP> p i ù <SEP> p i a n t a <GO> c h e <SEP> p i ù <SEP> d i <SEP> s é <SEP> l a <SEP> p i a n g e <SEP> s i <SEP> s p i g a n t a . <EOT> 


In [ ]:
!tar zcvf checkpoints.tar.gz checkpoints

checkpoints/
checkpoints/char-level_gen/
checkpoints/char-level_gen/ckpt-3.index
checkpoints/char-level_gen/ckpt-2.index
checkpoints/char-level_gen/ckpt-1.index
checkpoints/char-level_gen/ckpt-2.data-00000-of-00001
checkpoints/char-level_gen/ckpt-1.data-00000-of-00001
checkpoints/char-level_gen/checkpoint
checkpoints/char-level_gen/ckpt-3.data-00000-of-00001
checkpoints/char-level_gen/ckpt-4.data-00000-of-00001
checkpoints/char-level_gen/ckpt-4.index


In [ ]:
if 'google.colab' in str(get_ipython()):
    files.download('checkpoints.tar.gz')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>